# Реализация Encoder-Decoder

In [ ]:
import codecs
import numpy as np
import tensorflow as tf
print(tf.__version__)

## Загрузка датасета

In [10]:
data_fpath = 'rus.txt'
max_sentences = 10000

input_texts = []
target_texts = []
lines = codecs.open(data_fpath, 'r', encoding='utf8').readlines()[:max_sentences]
for line in lines:
    input_text, target_text, = line.split('\t')[:2]
    input_texts.append(input_text)
    target_texts.append(target_text)

## Подготовка словарей

In [11]:
def prepare_vocab(texts):
    vocab = sorted(set(''.join(texts)))
    vocab.append('<START>')
    vocab.append('<END>')
    vocab_size = len(vocab)
    char2idx = {u:i for i, u in enumerate(vocab)}
    idx2char = np.array(vocab)
    return vocab_size, char2idx, idx2char

INPUT_VOCAB_SIZE, input_char2idx, input_idx2char = prepare_vocab(input_texts)
TARGET_VOCAB_SIZE, target_char2idx, target_idx2char = prepare_vocab(target_texts)

## Подготовка обучающего датасета

In [12]:
input_texts_as_int = [[input_char2idx[c] for c in text] for text in input_texts]
target_texts_as_int = [[target_char2idx[c] for c in text] for text in target_texts]

encoder_input_seqs = [np.array(text) for text in input_texts_as_int]
decoder_input_seqs = []
decoder_target_seqs = []
for target_text in target_texts_as_int:
    decoder_input_seqs.append(np.array([target_char2idx['<START>']] + target_text))
    decoder_target_seqs.append(np.array(target_text + [target_char2idx['<END>']]))

## Паддинг цепочек

In [14]:
max_enc_seq_length = max([len(seq) for seq in encoder_input_seqs])
max_dec_seq_length = max([len(seq) for seq in decoder_input_seqs])

encoder_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(
    encoder_input_seqs,
    value=input_char2idx[' '],
    padding='post',
    maxlen=max_enc_seq_length)

decoder_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(
    decoder_input_seqs,
    value=target_char2idx[' '],
    padding='post',
    maxlen=max_dec_seq_length)

decoder_target_seqs = tf.keras.preprocessing.sequence.pad_sequences(
    decoder_target_seqs,
    value=target_char2idx[' '],
    padding='post',
    maxlen=max_dec_seq_length)

## Создание модели

In [16]:
H_SIZE = 256 # Размерность скрытого состояния LSTM
EMB_SIZE = 256 # размерность эмбеддингов (и для входных и для выходных цепочек)

class Encoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embed = tf.keras.layers.Embedding(INPUT_VOCAB_SIZE, EMB_SIZE)
        self.lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=False, return_state=True)
        
    def call(self, x):
        out = self.embed(x)
        _, h, c = self.lstm(out)
        state = (h, c)
        return state

class Decoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embed = tf.keras.layers.Embedding(TARGET_VOCAB_SIZE, EMB_SIZE)
        self.lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=True, return_state=True)
        self.fc = tf.keras.layers.Dense(TARGET_VOCAB_SIZE, activation='softmax')
        
    def call(self, x, init_state):
        out = self.embed(x)
        out, h, c = self.lstm(out, initial_state=init_state)
        out = self.fc(out)
        state = (h, c)
        return out, state

encoder_model = Encoder()
decoder_model = Decoder()

encoder_inputs = tf.keras.layers.Input(shape=(None,))
decoder_inputs = tf.keras.layers.Input(shape=(None,))

enc_state = encoder_model(encoder_inputs)
decoder_outputs, _ = decoder_model(decoder_inputs, enc_state)

seq2seq = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

## Обучение модели

In [17]:
BATCH_SIZE = 64
EPOCHS = 100

loss = tf.losses.SparseCategoricalCrossentropy()
seq2seq.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])
seq2seq.fit([encoder_input_seqs, decoder_input_seqs], decoder_target_seqs,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS)

Epoch 1/100
157/157 [==============================] - 11s 20ms/step - loss: 0.8445 - accuracy: 0.7936
Epoch 2/100
157/157 [==============================] - 3s 19ms/step - loss: 0.5398 - accuracy: 0.8443
Epoch 3/100
157/157 [==============================] - 3s 20ms/step - loss: 0.4717 - accuracy: 0.8623
Epoch 4/100
157/157 [==============================] - 3s 20ms/step - loss: 0.4249 - accuracy: 0.8751
Epoch 5/100
157/157 [==============================] - 3s 20ms/step - loss: 0.3885 - accuracy: 0.8867
Epoch 6/100
157/157 [==============================] - 3s 19ms/step - loss: 0.3592 - accuracy: 0.8951
Epoch 7/100
157/157 [==============================] - 3s 20ms/step - loss: 0.3344 - accuracy: 0.9023
Epoch 8/100
157/157 [==============================] - 3s 20ms/step - loss: 0.3133 - accuracy: 0.9083
Epoch 9/100
157/157 [==============================] - 3s 20ms/step - loss: 0.2941 - accuracy: 0.9137
Epoch 10/100
157/157 [==============================] - 3s 20ms/step - loss: 0.27

## Функция для инференса

In [18]:
def seq2seq_inference(input_seq):
    state = encoder_model(input_seq)

    target_seq = np.array([[target_char2idx['<START>']]])

    decoded_sentence = ''
    while True:
        output_tokens, state = decoder_model(target_seq, state)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_idx2char[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '<END>' or
           len(decoded_sentence) > max_dec_seq_length):
            break

        target_seq = np.array([[sampled_token_index]])

    return decoded_sentence

## Пример инференса

In [20]:
for seq_index in range(0, 10):
    input_seq = encoder_input_seqs[seq_index: seq_index + 1]
    decoded_sentence = seq2seq_inference(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Result sentence:', decoded_sentence)
    print('Target sentence:', target_texts[seq_index])

-
Input sentence: Go.
Result sentence: Идите.<END>
Target sentence: Марш!
-
Input sentence: Go.
Result sentence: Идите.<END>
Target sentence: Иди.
-
Input sentence: Go.
Result sentence: Идите.<END>
Target sentence: Идите.
-
Input sentence: Hi.
Result sentence: Здравствуйте.<END>
Target sentence: Здравствуйте.
-
Input sentence: Hi.
Result sentence: Здравствуйте.<END>
Target sentence: Привет!
-
Input sentence: Hi.
Result sentence: Здравствуйте.<END>
Target sentence: Хай.
-
Input sentence: Hi.
Result sentence: Здравствуйте.<END>
Target sentence: Здрасте.
-
Input sentence: Hi.
Result sentence: Здравствуйте.<END>
Target sentence: Здоро́во!
-
Input sentence: Run!
Result sentence: Бегите!<END>
Target sentence: Беги!
-
Input sentence: Run!
Result sentence: Бегите!<END>
Target sentence: Бегите!
